5.7 Exercise Problem 1
---

2232036001 권준형


5장 연습문제의 원본 코드

In [1]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

In [2]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

모델을 다시 정의한다.
모델은 t_u에 대한 2차 모델

In [3]:
def model(t_u, w2, w1, b):
    return w2 * t_u **2 + w1 * t_u + b

In [4]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

새로운 파라미터 w2, w1에 맞게 파라미터를 초기화한다.

In [5]:
w2 = torch.ones(())
w1 = torch.ones(())
b = torch.zeros(())

t_p = model(t_u, w2, w1, b)
t_p

tensor([1310.1901, 3180.7100, 3445.4399, 6789.5103, 3225.9900, 2440.1101,
        1183.1101,  497.0399, 2390.9600, 3708.5601, 4746.9600])

기존의 오차와 비교해보자.
tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000,
        21.8000, 48.4000, 60.4000, 68.4000])
기존의 오차에 비해 매우 큰 오차가 발생한다.

In [6]:
loss = loss_fn(t_p, t_c)
loss

tensor(11709471.)

loss 값 역시 tensor(1763.8846)와 비교하여 매우 큰 값이 발생하였다.

In [7]:
x = torch.ones(())
y = torch.ones(3,1)
z = torch.ones(1,3)
a = torch.ones(2, 1, 1)
print(f"shapes: x: {x.shape}, y: {y.shape}")
print(f"        z: {z.shape}, a: {a.shape}")
print("x * y:", (x * y).shape)
print("y * z:", (y * z).shape)
print("y * z * a:", (y * z * a).shape)

shapes: x: torch.Size([]), y: torch.Size([3, 1])
        z: torch.Size([1, 3]), a: torch.Size([2, 1, 1])
x * y: torch.Size([3, 1])
y * z: torch.Size([3, 3])
y * z * a: torch.Size([2, 3, 3])


늘어난 파라미터에 맞게 손실 변화율을 설정한다.
w2, w1 손실 변화율 설정

w2, w1, b에 대한 미분 설정

In [8]:
delta = 0.1

loss_rate_of_change_w2 = \
    (loss_fn(model(t_u, w2 + delta, w1, b), t_c) - 
     loss_fn(model(t_u, w2 - delta, w1, b), t_c)) / (2.0 * delta)

loss_rate_of_change_w1 = \
    (loss_fn(model(t_u, w2, w1 + delta, b), t_c) - 
     loss_fn(model(t_u, w2, w1 - delta, b), t_c)) / (2.0 * delta)

In [9]:
learning_rate = 1e-2

w2 = w2 - learning_rate * loss_rate_of_change_w2
w1 = w1 - learning_rate * loss_rate_of_change_w1

In [10]:
loss_rate_of_change_b = \
    (loss_fn(model(t_u, w2, w1, b + delta), t_c) - 
     loss_fn(model(t_u, w2, w1, b - delta), t_c)) / (2.0 * delta)

b = b - learning_rate * loss_rate_of_change_b

In [11]:
def dloss_fn(t_p, t_c):
    dsq_diffs = 2 * (t_p - t_c) / t_p.size(0)
    return dsq_diffs

모델에 미분 함수를 적용한다.
각 파라미터에 대하여 편미분을 적용한 함수를 미분 함수로 정의한다.

In [12]:
def dmodel_dw2(t_u, w2, w1, b):
    return 2 * t_u

In [13]:
def dmodel_dw1(t_u, w2, w1, b):
    return t_u

In [14]:
def dmodel_db(t_u, w2, w1, b):
    return 1.0

미분함수들을 합쳐서 모델 파라미터에 대한 손실값의 미분을 반환하는 경사 함수를 정의한다.

In [15]:
def grad_fn(t_u, t_c, t_p, w2, w1, b):
    dloss_dtp = dloss_fn(t_p, t_c)
    dloss_dw2 = dloss_dtp * dmodel_dw2(t_u, w2, w1, b)
    dloss_dw1 = dloss_dtp * dmodel_dw1(t_u, w2, w1, b)
    dloss_db = dloss_dtp * dmodel_db(t_u, w2, w1, b)
    return torch.stack([dloss_dw2.sum(), dloss_dw1.sum(), dloss_db.sum()])

모델 적합을 위한 훈련 루프를 만든다.
파라미터가 3개로 늘어났으므로, params는 w2,w1,b 세 파라미터를 갖도록 선언한다.
훈련 루프를 실행할때는 세 변수에 대한 초깃값을 선언한다.

In [16]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        w2, w1, b = params

        t_p = model(t_u, w2, w1, b)  
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w2, w1, b) 

        params = params - learning_rate * grad

        print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [18]:
training_loop(
    n_epochs = 100, 
    learning_rate = 1e-2, 
    params = torch.tensor([1.0, 1.0, 0.0]), 
    t_u = t_u, 
    t_c = t_c)

Epoch 1, Loss 11709471.000000
Epoch 2, Loss 608308431945728.000000
Epoch 3, Loss 31567049342930626019328.000000
Epoch 4, Loss 1638114432853846173108251131904.000000
Epoch 5, Loss inf
Epoch 6, Loss inf
Epoch 7, Loss inf
Epoch 8, Loss inf
Epoch 9, Loss inf
Epoch 10, Loss inf
Epoch 11, Loss inf
Epoch 12, Loss nan
Epoch 13, Loss nan
Epoch 14, Loss nan
Epoch 15, Loss nan
Epoch 16, Loss nan
Epoch 17, Loss nan
Epoch 18, Loss nan
Epoch 19, Loss nan
Epoch 20, Loss nan
Epoch 21, Loss nan
Epoch 22, Loss nan
Epoch 23, Loss nan
Epoch 24, Loss nan
Epoch 25, Loss nan
Epoch 26, Loss nan
Epoch 27, Loss nan
Epoch 28, Loss nan
Epoch 29, Loss nan
Epoch 30, Loss nan
Epoch 31, Loss nan
Epoch 32, Loss nan
Epoch 33, Loss nan
Epoch 34, Loss nan
Epoch 35, Loss nan
Epoch 36, Loss nan
Epoch 37, Loss nan
Epoch 38, Loss nan
Epoch 39, Loss nan
Epoch 40, Loss nan
Epoch 41, Loss nan
Epoch 42, Loss nan
Epoch 43, Loss nan
Epoch 44, Loss nan
Epoch 45, Loss nan
Epoch 46, Loss nan
Epoch 47, Loss nan
Epoch 48, Loss nan
Epoc

tensor([nan, nan, nan])

발산한다.

learningrate를 조정하면서 다시 학습을 시켜본다.

In [21]:
training_loop(
    n_epochs = 100, 
    learning_rate = 1e-12, 
    params = torch.tensor([1.0, 1.0, 0.0]), 
    t_u = t_u, 
    t_c = t_c)

Epoch 1, Loss 11709471.000000
Epoch 2, Loss 11709455.000000
Epoch 3, Loss 11709439.000000
Epoch 4, Loss 11709423.000000
Epoch 5, Loss 11709407.000000
Epoch 6, Loss 11709390.000000
Epoch 7, Loss 11709373.000000
Epoch 8, Loss 11709357.000000
Epoch 9, Loss 11709340.000000
Epoch 10, Loss 11709323.000000
Epoch 11, Loss 11709307.000000
Epoch 12, Loss 11709290.000000
Epoch 13, Loss 11709273.000000
Epoch 14, Loss 11709255.000000
Epoch 15, Loss 11709239.000000
Epoch 16, Loss 11709223.000000
Epoch 17, Loss 11709204.000000
Epoch 18, Loss 11709188.000000
Epoch 19, Loss 11709172.000000
Epoch 20, Loss 11709155.000000
Epoch 21, Loss 11709139.000000
Epoch 22, Loss 11709122.000000
Epoch 23, Loss 11709105.000000
Epoch 24, Loss 11709089.000000
Epoch 25, Loss 11709073.000000
Epoch 26, Loss 11709056.000000
Epoch 27, Loss 11709039.000000
Epoch 28, Loss 11709023.000000
Epoch 29, Loss 11709006.000000
Epoch 30, Loss 11708989.000000
Epoch 31, Loss 11708972.000000
Epoch 32, Loss 11708955.000000
Epoch 33, Loss 11

tensor([ 9.9993e-01,  9.9996e-01, -5.9640e-07])

정규화를 시킨 뒤 다시 학습을 진행한다.

In [22]:
t_un = 0.1 * t_u

In [24]:
training_loop(
    n_epochs = 100, 
    learning_rate = 1e-12, 
    params = torch.tensor([1.0, 1.0, 0.0]), 
    t_u = t_un, # <1>
    t_c = t_c)

Epoch 1, Loss 675.794373
Epoch 2, Loss 675.794373
Epoch 3, Loss 675.794373
Epoch 4, Loss 675.794373
Epoch 5, Loss 675.794373
Epoch 6, Loss 675.794373
Epoch 7, Loss 675.794373
Epoch 8, Loss 675.794373
Epoch 9, Loss 675.794373
Epoch 10, Loss 675.794373
Epoch 11, Loss 675.794373
Epoch 12, Loss 675.794373
Epoch 13, Loss 675.794373
Epoch 14, Loss 675.794373
Epoch 15, Loss 675.794373
Epoch 16, Loss 675.794373
Epoch 17, Loss 675.794373
Epoch 18, Loss 675.794373
Epoch 19, Loss 675.794373
Epoch 20, Loss 675.794373
Epoch 21, Loss 675.794373
Epoch 22, Loss 675.794373
Epoch 23, Loss 675.794373
Epoch 24, Loss 675.794373
Epoch 25, Loss 675.794373
Epoch 26, Loss 675.794373
Epoch 27, Loss 675.794373
Epoch 28, Loss 675.794373
Epoch 29, Loss 675.794373
Epoch 30, Loss 675.794373
Epoch 31, Loss 675.794373
Epoch 32, Loss 675.794373
Epoch 33, Loss 675.794373
Epoch 34, Loss 675.794373
Epoch 35, Loss 675.794373
Epoch 36, Loss 675.794373
Epoch 37, Loss 675.794373
Epoch 38, Loss 675.794373
Epoch 39, Loss 675.79

tensor([ 1.0000e+00,  1.0000e+00, -4.8176e-09])

In [29]:
params = training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-8, 
    params = torch.tensor([1.0, 1.0, 0.0]), 
    t_u = t_un, 
    t_c = t_c)

params

Epoch 1, Loss 675.794373
Epoch 2, Loss 675.783752
Epoch 3, Loss 675.773193
Epoch 4, Loss 675.762695
Epoch 5, Loss 675.752075
Epoch 6, Loss 675.741577
Epoch 7, Loss 675.730896
Epoch 8, Loss 675.720398
Epoch 9, Loss 675.709900
Epoch 10, Loss 675.699341
Epoch 11, Loss 675.688721
Epoch 12, Loss 675.678223
Epoch 13, Loss 675.667725
Epoch 14, Loss 675.657043
Epoch 15, Loss 675.646606
Epoch 16, Loss 675.635986
Epoch 17, Loss 675.625549
Epoch 18, Loss 675.615051
Epoch 19, Loss 675.604431
Epoch 20, Loss 675.593811
Epoch 21, Loss 675.583313
Epoch 22, Loss 675.572815
Epoch 23, Loss 675.562195
Epoch 24, Loss 675.551636
Epoch 25, Loss 675.541077
Epoch 26, Loss 675.530518
Epoch 27, Loss 675.519836
Epoch 28, Loss 675.509338
Epoch 29, Loss 675.498779
Epoch 30, Loss 675.488220
Epoch 31, Loss 675.477783
Epoch 32, Loss 675.467224
Epoch 33, Loss 675.456726
Epoch 34, Loss 675.446045
Epoch 35, Loss 675.435486
Epoch 36, Loss 675.424988
Epoch 37, Loss 675.414368
Epoch 38, Loss 675.403870
Epoch 39, Loss 675.39

Epoch 709, Loss 668.367920
Epoch 710, Loss 668.357666
Epoch 711, Loss 668.347168
Epoch 712, Loss 668.336731
Epoch 713, Loss 668.326355
Epoch 714, Loss 668.315979
Epoch 715, Loss 668.305664
Epoch 716, Loss 668.295166
Epoch 717, Loss 668.284851
Epoch 718, Loss 668.274414
Epoch 719, Loss 668.264099
Epoch 720, Loss 668.253479
Epoch 721, Loss 668.243225
Epoch 722, Loss 668.232910
Epoch 723, Loss 668.222473
Epoch 724, Loss 668.212097
Epoch 725, Loss 668.201721
Epoch 726, Loss 668.191284
Epoch 727, Loss 668.180908
Epoch 728, Loss 668.170471
Epoch 729, Loss 668.160156
Epoch 730, Loss 668.149780
Epoch 731, Loss 668.139404
Epoch 732, Loss 668.128967
Epoch 733, Loss 668.118591
Epoch 734, Loss 668.108154
Epoch 735, Loss 668.097839
Epoch 736, Loss 668.087402
Epoch 737, Loss 668.076965
Epoch 738, Loss 668.066650
Epoch 739, Loss 668.056213
Epoch 740, Loss 668.045837
Epoch 741, Loss 668.035461
Epoch 742, Loss 668.025146
Epoch 743, Loss 668.014709
Epoch 744, Loss 668.004333
Epoch 745, Loss 667.993958
E

Epoch 1435, Loss 660.850464
Epoch 1436, Loss 660.840271
Epoch 1437, Loss 660.829895
Epoch 1438, Loss 660.819458
Epoch 1439, Loss 660.809326
Epoch 1440, Loss 660.798889
Epoch 1441, Loss 660.788574
Epoch 1442, Loss 660.778259
Epoch 1443, Loss 660.768005
Epoch 1444, Loss 660.757568
Epoch 1445, Loss 660.747314
Epoch 1446, Loss 660.737000
Epoch 1447, Loss 660.726624
Epoch 1448, Loss 660.716370
Epoch 1449, Loss 660.705994
Epoch 1450, Loss 660.695740
Epoch 1451, Loss 660.685486
Epoch 1452, Loss 660.675049
Epoch 1453, Loss 660.664795
Epoch 1454, Loss 660.654541
Epoch 1455, Loss 660.644104
Epoch 1456, Loss 660.633911
Epoch 1457, Loss 660.623596
Epoch 1458, Loss 660.613281
Epoch 1459, Loss 660.602966
Epoch 1460, Loss 660.592712
Epoch 1461, Loss 660.582397
Epoch 1462, Loss 660.571960
Epoch 1463, Loss 660.561646
Epoch 1464, Loss 660.551331
Epoch 1465, Loss 660.541077
Epoch 1466, Loss 660.530762
Epoch 1467, Loss 660.520447
Epoch 1468, Loss 660.510132
Epoch 1469, Loss 660.499878
Epoch 1470, Loss 660

Epoch 2208, Loss 652.937317
Epoch 2209, Loss 652.927124
Epoch 2210, Loss 652.916931
Epoch 2211, Loss 652.906799
Epoch 2212, Loss 652.896790
Epoch 2213, Loss 652.886475
Epoch 2214, Loss 652.876465
Epoch 2215, Loss 652.866272
Epoch 2216, Loss 652.856079
Epoch 2217, Loss 652.845947
Epoch 2218, Loss 652.835693
Epoch 2219, Loss 652.825623
Epoch 2220, Loss 652.815491
Epoch 2221, Loss 652.805359
Epoch 2222, Loss 652.795227
Epoch 2223, Loss 652.785034
Epoch 2224, Loss 652.774841
Epoch 2225, Loss 652.764648
Epoch 2226, Loss 652.754639
Epoch 2227, Loss 652.744385
Epoch 2228, Loss 652.734375
Epoch 2229, Loss 652.724060
Epoch 2230, Loss 652.713989
Epoch 2231, Loss 652.703857
Epoch 2232, Loss 652.693726
Epoch 2233, Loss 652.683533
Epoch 2234, Loss 652.673401
Epoch 2235, Loss 652.663208
Epoch 2236, Loss 652.653137
Epoch 2237, Loss 652.642944
Epoch 2238, Loss 652.632812
Epoch 2239, Loss 652.622620
Epoch 2240, Loss 652.612549
Epoch 2241, Loss 652.602295
Epoch 2242, Loss 652.592224
Epoch 2243, Loss 652

Epoch 3023, Loss 644.692322
Epoch 3024, Loss 644.682129
Epoch 3025, Loss 644.672119
Epoch 3026, Loss 644.662048
Epoch 3027, Loss 644.651978
Epoch 3028, Loss 644.641968
Epoch 3029, Loss 644.631897
Epoch 3030, Loss 644.621765
Epoch 3031, Loss 644.611755
Epoch 3032, Loss 644.601624
Epoch 3033, Loss 644.591431
Epoch 3034, Loss 644.581482
Epoch 3035, Loss 644.571228
Epoch 3036, Loss 644.561279
Epoch 3037, Loss 644.551147
Epoch 3038, Loss 644.541138
Epoch 3039, Loss 644.531006
Epoch 3040, Loss 644.520935
Epoch 3041, Loss 644.510742
Epoch 3042, Loss 644.500793
Epoch 3043, Loss 644.490662
Epoch 3044, Loss 644.480530
Epoch 3045, Loss 644.470642
Epoch 3046, Loss 644.460388
Epoch 3047, Loss 644.450378
Epoch 3048, Loss 644.440369
Epoch 3049, Loss 644.430176
Epoch 3050, Loss 644.420044
Epoch 3051, Loss 644.410034
Epoch 3052, Loss 644.399963
Epoch 3053, Loss 644.389832
Epoch 3054, Loss 644.379700
Epoch 3055, Loss 644.369629
Epoch 3056, Loss 644.359619
Epoch 3057, Loss 644.349426
Epoch 3058, Loss 644

Epoch 3858, Loss 636.360291
Epoch 3859, Loss 636.350342
Epoch 3860, Loss 636.340515
Epoch 3861, Loss 636.330505
Epoch 3862, Loss 636.320496
Epoch 3863, Loss 636.310669
Epoch 3864, Loss 636.300781
Epoch 3865, Loss 636.290771
Epoch 3866, Loss 636.280945
Epoch 3867, Loss 636.270996
Epoch 3868, Loss 636.261108
Epoch 3869, Loss 636.251099
Epoch 3870, Loss 636.241211
Epoch 3871, Loss 636.231323
Epoch 3872, Loss 636.221313
Epoch 3873, Loss 636.211487
Epoch 3874, Loss 636.201599
Epoch 3875, Loss 636.191650
Epoch 3876, Loss 636.181885
Epoch 3877, Loss 636.171814
Epoch 3878, Loss 636.161865
Epoch 3879, Loss 636.152100
Epoch 3880, Loss 636.142090
Epoch 3881, Loss 636.132202
Epoch 3882, Loss 636.122192
Epoch 3883, Loss 636.112366
Epoch 3884, Loss 636.102417
Epoch 3885, Loss 636.092407
Epoch 3886, Loss 636.082581
Epoch 3887, Loss 636.072693
Epoch 3888, Loss 636.062744
Epoch 3889, Loss 636.052917
Epoch 3890, Loss 636.043030
Epoch 3891, Loss 636.033081
Epoch 3892, Loss 636.023193
Epoch 3893, Loss 636

Epoch 4640, Loss 628.640442
Epoch 4641, Loss 628.630615
Epoch 4642, Loss 628.620789
Epoch 4643, Loss 628.610962
Epoch 4644, Loss 628.601135
Epoch 4645, Loss 628.591309
Epoch 4646, Loss 628.581421
Epoch 4647, Loss 628.571594
Epoch 4648, Loss 628.561707
Epoch 4649, Loss 628.551941
Epoch 4650, Loss 628.542053
Epoch 4651, Loss 628.532166
Epoch 4652, Loss 628.522400
Epoch 4653, Loss 628.512634
Epoch 4654, Loss 628.502686
Epoch 4655, Loss 628.492981
Epoch 4656, Loss 628.483032
Epoch 4657, Loss 628.473206
Epoch 4658, Loss 628.463318
Epoch 4659, Loss 628.453552
Epoch 4660, Loss 628.443726
Epoch 4661, Loss 628.433838
Epoch 4662, Loss 628.424072
Epoch 4663, Loss 628.414185
Epoch 4664, Loss 628.404358
Epoch 4665, Loss 628.394592
Epoch 4666, Loss 628.384705
Epoch 4667, Loss 628.374756
Epoch 4668, Loss 628.364990
Epoch 4669, Loss 628.355225
Epoch 4670, Loss 628.345276
Epoch 4671, Loss 628.335510
Epoch 4672, Loss 628.325623
Epoch 4673, Loss 628.315735
Epoch 4674, Loss 628.305969
Epoch 4675, Loss 628

tensor([ 0.9726,  0.9863, -0.0024])

a. 이 모델을 다루기 위해 훈련 루프의 어떤 부분을 변경해야 하나?
---
Ans) 훈련 루프에서는 params 초기값을 2개의 텐서를 입력받던 부분을 3개의 텐서를 입력받도록 변경하였다.

b. 모델 변경과 무관한 부분은 어디인가?
---
Ans) 훈련 루프에서는 params 입력 부분 외에는 동일하게 사용하였다.

c. 훈련 후 손실이 증가하는가 감소하는가?
--
Ans) 훈련 후 손실이 (매우 크게) 증가하였다.

d. 실제 결과가 좋아졌나 나빠졌나?
---
Ans) 교재에 나온 학습률을 적용하면 발산한다. 교재보다 학습률을 크게 낮추면 수렴하지만 손실이 여전히 크게 나온다. 결과는 좋아졌다고 말하기 어렵다.
본 예제는 섭씨와 화씨 온도 데이터의 관계인 선형 모델을 모사하기 때문에, 2차 모델을 사용하더라도 결과가 좋아지지 않은 것이라고 생각한다.